In [ ]:
!python -V

In [2]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet')

In [ ]:
len(df_train), len(df_val)

In [8]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')


In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
from sklearn.pipeline import make_pipeline


pipeline = make_pipeline(DictVectorizer(), LinearRegression())
pipeline.fit(train_dicts, y_train)
y_pred = pipeline.predict(val_dicts)

mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
sns.histplot(y_pred, kde=True, stat="density", color='blue', bins=25, label='prediction')
sns.histplot(y_val, kde=True, stat="density", color='orange', bins=40, label='actual')

plt.legend()

In [11]:
with open('lin_reg.bin', 'wb') as f_out:
    pickle.dump(pipeline, f_out)

In [ ]:
with open('lin_reg.bin', 'rb') as f_in:
    pipeline_loaded = pickle.load(f_in)

y_pred = pipeline_loaded.predict(val_dicts)
mean_squared_error(y_val, y_pred, squared=False)